# Inspect and Summarize Silver SPY Parquet Files

This notebook helps you inspect and summarize your silver-layer SPY 1-minute bar data. It loads, displays, and summarizes the cleaned data ready for analysis or visualization.

In [1]:
# Import Required Libraries
import os
import pandas as pd
import duckdb

In [2]:
# Plotly renderer setup for VS Code + quick sanity check
import plotly.io as pio
try:
    pio.renderers.default = 'vscode'  # Best for VS Code Jupyter
except Exception:
    try:
        pio.renderers.default = 'plotly_mimetype'  # Fallback MIME renderer
    except Exception:
        pio.renderers.default = 'notebook_connected'  # Last resort
print(f"Plotly renderer: {pio.renderers.default}")

Plotly renderer: vscode


In [3]:
# Set up path to silver SPY data
silver_dir = os.path.join(
    os.path.dirname(os.path.dirname(os.path.abspath("../scripts/inspect_silver_spy.py"))),
    "data", "silver", "SPY"
)

In [4]:
# Sanity check plot
import plotly.graph_objects as go
fig__sanity = go.Figure(data=[go.Scatter(x=[0,1,2], y=[0,1,0], mode='lines+markers', name='sanity')])
fig__sanity.update_layout(title='Plotly Sanity Check')
fig__sanity.show()

In [5]:
# List and load parquet files
files = sorted([f for f in os.listdir(silver_dir) if f.endswith(".parquet")])

if not files:
    raise FileNotFoundError(f"No parquet files found in {silver_dir}")

In [6]:
# Inspect a sample parquet file
sample_file = os.path.join(silver_dir, files[0])
df = pd.read_parquet(sample_file)

print(f"Sample file: {sample_file}")
display(df.head())
display(df.info())
print("\nColumns:", df.columns.tolist())

Sample file: /home/agoodman/workspace/antman_data/data/silver/SPY/spy_1min_2025-01-02.parquet


,close,high,low,num_trades,open,volume,vwap
t,,,,,,,
2025-01-02 14:30:00+00:00,589.45,589.45,588.4600,16472,589.39,1300579,588.918280
2025-01-02 14:31:00+00:00,589.57,589.60,589.0400,2160,589.45,175662,589.334934
2025-01-02 14:32:00+00:00,589.09,589.65,589.0319,1542,589.49,125032,589.381349
2025-01-02 14:33:00+00:00,588.01,589.18,587.9800,2396,589.09,132101,588.624495
2025-01-02 14:34:00+00:00,587.83,588.52,587.8100,1916,587.99,145261,588.180358


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 391 entries, 2025-01-02 14:30:00+00:00 to 2025-01-02 21:00:00+00:00
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   close       391 non-null    float64
 1   high        391 non-null    float64
 2   low         391 non-null    float64
 3   num_trades  391 non-null    int64  
 4   open        391 non-null    float64
 5   volume      391 non-null    int64  
 6   vwap        391 non-null    float64
dtypes: float64(5), int64(2)
memory usage: 24.4 KB


None


Columns: ['close', 'high', 'low', 'num_trades', 'open', 'volume', 'vwap']


In [7]:
# Summarize all parquet files (row counts, date range) — fast via DuckDB
import duckdb

glob_path = os.path.join(silver_dir, "*.parquet")
# Count rows across all Parquet files efficiently
with duckdb.connect() as con:
    (total_rows,) = con.execute(
        "SELECT COUNT(*) FROM read_parquet(?, union_by_name=true)", [glob_path]
    ).fetchone()

# Derive available dates from filenames (no data load)
dates = sorted(f.split("_")[-1].replace(".parquet", "") for f in files)
print(f"Total files: {len(files)}")
print(f"Total rows: {total_rows}")
print(f"Date range: {min(dates)} to {max(dates)}")

Total files: 149
Total rows: 58094
Date range: 2025-01-02 to 2025-08-07


## Visualizing SPY 1-Minute Bars with Plotly

The following cells demonstrate how to create interactive candlestick and volume charts using Plotly. These are standard visualizations for financial time series data.

In [8]:
# Plotly Candlestick Chart for SPY 1-Minute Bars
import plotly.graph_objects as go

# Ensure 'df' is loaded from a sample file (already done above)
fig = go.Figure(data=[
    go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='SPY 1-min'
    )
])
fig.update_layout(
    title='SPY 1-Minute Candlestick Chart',
    xaxis_title='Time',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False
)
fig.show()

In [9]:
# Plotly Volume Bar Chart for SPY 1-Minute Bars
import plotly.express as px

fig_vol = px.bar(
    df,
    x=df.index,
    y='volume',
    labels={'x': 'Time', 'volume': 'Volume'},
    title='SPY 1-Minute Volume'
)
fig_vol.show()

## 1-Day Intraday Plotly Dashboard (SPY)

This section plots a rich, interactive dashboard for a single trading day using Plotly:
- Candlesticks with Moving Averages (9/21/50) and VWAP
- Volume bars colored by up/down
- Hover tooltips and unified crosshair
- Optional highlight of Regular Trading Hours (RTH)

Set `target_date` (YYYY-MM-DD) to choose the session. If not set, it will default to the date from the loaded sample.

In [10]:
# 1-Day Intraday Plotly Dashboard
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- Configuration ---
# Choose a date in YYYY-MM-DD. If None, use the date of the currently loaded df.
target_date = None  # e.g. '2025-01-02'

# --- Prepare data ---
if 'df' not in globals():
    raise RuntimeError("DataFrame 'df' not found. Run the earlier cells that load a sample parquet file.")

# Ensure index is DatetimeIndex and tz-aware
if not isinstance(df.index, pd.DatetimeIndex):
    df.index = pd.to_datetime(df.index, utc=True)
elif df.index.tz is None:
    df.index = df.index.tz_localize('UTC')

# Determine the date to plot
if target_date is None:
    target_date = df.index[0].date().isoformat()

day_mask = df.index.date == pd.to_datetime(target_date).date()
day = df.loc[day_mask].copy()
if day.empty:
    raise ValueError(f"No rows found for {target_date}. Make sure this date exists in your silver data.")

# Compute indicators
for window in (9, 21, 50):
    day[f"sma_{window}"] = day['close'].rolling(window, min_periods=1).mean()

# Intraday VWAP (cumulative typical price * volume / cumulative volume)
typical = (day['high'] + day['low'] + day['close']) / 3.0
cum_v = day['volume'].cumsum().replace(0, np.nan)
day['vwap_intraday'] = (typical * day['volume']).cumsum() / cum_v

# Color volume bars green/red depending on close >= open
vol_colors = np.where(day['close'] >= day['open'], 'rgba(0,150,0,0.6)', 'rgba(200,0,0,0.6)')

# Build figure with two rows: price + volume
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03,
    row_heights=[0.72, 0.28],
    specs=[[{"secondary_y": False}], [{"secondary_y": False}]]
)

# Candlesticks
fig.add_trace(
    go.Candlestick(
        x=day.index,
        open=day['open'], high=day['high'], low=day['low'], close=day['close'],
        increasing_line_color='#26a69a', decreasing_line_color='#ef5350',
        name='OHLC'
    ), row=1, col=1
)

# Moving averages
fig.add_trace(go.Scatter(x=day.index, y=day['sma_9'],  name='SMA 9',  line=dict(color='#42a5f5', width=1.5)), row=1, col=1)
fig.add_trace(go.Scatter(x=day.index, y=day['sma_21'], name='SMA 21', line=dict(color='#ab47bc', width=1.5)), row=1, col=1)
fig.add_trace(go.Scatter(x=day.index, y=day['sma_50'], name='SMA 50', line=dict(color='#ffa726', width=1.5)), row=1, col=1)

# VWAP
fig.add_trace(go.Scatter(x=day.index, y=day['vwap_intraday'], name='VWAP', line=dict(color='#8d6e63', width=2, dash='dot')), row=1, col=1)

# Volume bars
fig.add_trace(
    go.Bar(x=day.index, y=day['volume'], marker_color=vol_colors, name='Volume'),
    row=2, col=1
)

# Layout tweaks for beauty and readability
fig.update_layout(
    title=f"SPY Intraday Dashboard — {target_date}",
    template='plotly_white',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    margin=dict(l=40, r=20, t=60, b=40),
    hovermode='x unified',
)

fig.update_xaxes(
    showgrid=True, gridcolor='rgba(200,200,200,0.2)',
    rangeslider_visible=False, type='date'
)
fig.update_yaxes(showgrid=True, gridcolor='rgba(200,200,200,0.2)', title_text='Price', row=1, col=1)
fig.update_yaxes(showgrid=True, gridcolor='rgba(200,200,200,0.2)', title_text='Volume', row=2, col=1)

# Optional: highlight Regular Trading Hours (9:30-16:00 ET)
try:
    import pytz
    ny = pytz.timezone('America/New_York')
    start_rth = pd.Timestamp(target_date + ' 09:30', tz=ny).astimezone(pytz.UTC)
    end_rth   = pd.Timestamp(target_date + ' 16:00', tz=ny).astimezone(pytz.UTC)
    fig.add_vrect(x0=start_rth, x1=end_rth, fillcolor='rgba(33,150,243,0.08)', line_width=0, layer='below', row=1, col=1)
    fig.add_vrect(x0=start_rth, x1=end_rth, fillcolor='rgba(33,150,243,0.08)', line_width=0, layer='below', row=2, col=1)
except Exception:
    pass

fig.show()

print("Tip: Set target_date = 'YYYY-MM-DD' above and re-run to pick a different session.")

Tip: Set target_date = 'YYYY-MM-DD' above and re-run to pick a different session.


## Momentum/Scalping Indicators + Interactive Date Picker

This section adds common momentum/scalping indicators and an interactive date selector:
- Overlays: EMA(9/21/50), Bollinger Bands (20, 2), VWAP (intraday)
- Volume: colored up/down
- Momentum panel: RSI(14) and MACD(12,26,9)
- Interactive controls: pick any available trading date and export as HTML/PNG

Use the widget to select a date and the figure will update automatically.

In [ ]:
# Interactive Date Picker + Technical Analysis Dashboard (EMA, Bollinger, RSI, MACD)
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display
import duckdb

# Ensure silver_dir and files are available, even if earlier cells weren't run
if 'silver_dir' not in globals():
    silver_dir = os.path.join(
        os.path.dirname(os.path.dirname(os.path.abspath("../scripts/inspect_silver_spy.py"))),
        "data", "silver", "SPY"
    )
try:
    files  # noqa: F821 - merely checking existence
except NameError:
    files = sorted([f for f in os.listdir(silver_dir) if f.endswith(".parquet")])

# Collect available dates from silver files
available_dates = sorted({f.split('_')[-1].replace('.parquet', '') for f in files})
if not available_dates:
    raise RuntimeError("No silver files found to populate the date picker.")

# Widgets
date_dd = widgets.Dropdown(options=available_dates, value=available_dates[0], description='Date:', layout=widgets.Layout(width='260px'))
export_html_btn = widgets.Button(description='Export HTML', tooltip='Save chart to charts/ as HTML', icon='download')
export_png_btn = widgets.Button(description='Export PNG', tooltip='Save chart to charts/ as PNG (requires kaleido)', icon='file')
status_lbl = widgets.Label('Select a date to render the dashboard...')
use_duckdb_chk = widgets.Checkbox(value=True, description='DuckDB load', tooltip='Load day directly via DuckDB from Parquet')
out = widgets.Output()

_last = {'fig': None, 'date': None}

# Helpers
def _compute_indicators(d: pd.DataFrame) -> pd.DataFrame:
    d = d.copy()
    # EMAs
    for span in (9, 21, 50):
        d[f'ema_{span}'] = d['close'].ewm(span=span, adjust=False).mean()
    # Bollinger Bands (20, 2)
    mid = d['close'].rolling(20, min_periods=1).mean()
    std = d['close'].rolling(20, min_periods=1).std()
    d['bb_mid'] = mid
    d['bb_upper'] = mid + 2 * std
    d['bb_lower'] = mid - 2 * std
    # RSI(14) - Wilder
    delta = d['close'].diff()
    gain = np.where(delta > 0, delta, 0.0)
    loss = np.where(delta < 0, -delta, 0.0)
    roll_up = pd.Series(gain, index=d.index).ewm(alpha=1/14, adjust=False).mean()
    roll_down = pd.Series(loss, index=d.index).ewm(alpha=1/14, adjust=False).mean()
    rs = roll_up / (roll_down + 1e-9)
    d['rsi_14'] = 100 - (100 / (1 + rs))
    # MACD(12,26,9)
    ema12 = d['close'].ewm(span=12, adjust=False).mean()
    ema26 = d['close'].ewm(span=26, adjust=False).mean()
    d['macd'] = ema12 - ema26
    d['macd_signal'] = d['macd'].ewm(span=9, adjust=False).mean()
    d['macd_hist'] = d['macd'] - d['macd_signal']
    return d

def _load_day(date_str: str) -> pd.DataFrame:
    """Load a single trading day from Parquet.
    When DuckDB is enabled, filter by date parsed from filename and create a unified 'ts' column.
    Fallback to pandas if needed. Ensures tz-aware, ms-rounded, sorted DatetimeIndex.
    """
    d = None
    if use_duckdb_chk.value:
        glob_path = os.path.join(silver_dir, "*.parquet")
        try:
            with duckdb.connect() as con:
                # Robust: filter on date parsed from filename; also project a unified timestamp column 'ts'
                sql = """
                    SELECT
                        *,
                        COALESCE(utc_timestamp, "timestamp", datetime) AS ts
                    FROM read_parquet(?, union_by_name=true, filename=true)
                    WHERE regexp_extract(filename, '(\\d{4}-\\d{2}-\\d{2})', 1) = ?
                    ORDER BY ts NULLS LAST
                """
                d = con.execute(sql, [glob_path, date_str]).df()
        except Exception:
            d = None
    # Pandas fallback if DuckDB path failed/empty or checkbox off
    if d is None or d.empty:
        path = os.path.join(silver_dir, f"spy_1min_{date_str}.parquet")
        if os.path.exists(path):
            d = pd.read_parquet(path)
        else:
            candidates = [f for f in files if date_str in f] if 'files' in globals() else []
            if candidates:
                d = pd.read_parquet(os.path.join(silver_dir, candidates[0]))
            else:
                return pd.DataFrame()

    # Normalize index to UTC DateTimeIndex, round to ms, sort
    idx_col = 'ts' if 'ts' in d.columns else next((c for c in ("utc_timestamp", "timestamp", "datetime", "index") if c in d.columns), None)
    if idx_col is not None:
        d = d.set_index(idx_col)
    d.index = pd.to_datetime(d.index, utc=True, errors='coerce')
    if d.index.tz is None:
        d.index = d.index.tz_localize('UTC')
    try:
        d.index = d.index.round('ms')
    except Exception:
        pass
    d = d.sort_index()

    # Ensure required columns exist
    required = ("open", "high", "low", "close", "volume")
    missing = [c for c in required if c not in d.columns]
    if missing:
        raise KeyError(f"Missing columns required for chart: {missing}")
    # Clean helper columns
    for c in ('filename', 'ts'):
        if c in d.columns:
            try:
                d = d.drop(columns=[c])
            except Exception:
                pass
    return d

def _build_fig(d: pd.DataFrame, date_str: str) -> go.Figure:
    d = _compute_indicators(d)
    vol_colors = np.where(d['close'] >= d['open'], 'rgba(0,150,0,0.6)', 'rgba(200,0,0,0.6)')

    fig = make_subplots(
        rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.03,
        row_heights=[0.58, 0.18, 0.24],
        specs=[[{"secondary_y": False}],
               [{"secondary_y": False}],
               [{"secondary_y": True}]]
    )

    # Row 1: Price with overlays
    fig.add_trace(go.Candlestick(x=d.index, open=d['open'], high=d['high'], low=d['low'], close=d['close'],
                                 increasing_line_color='#26a69a', decreasing_line_color='#ef5350', name='OHLC',
                                 showlegend=False), row=1, col=1)
    for span, color in [(9, '#42a5f5'), (21, '#ab47bc'), (50, '#ffa726')]:
        fig.add_trace(go.Scatter(x=d.index, y=d[f'ema_{span}'], name=f'EMA {span}', line=dict(width=1.4, color=color)), row=1, col=1)
    # Bollinger
    fig.add_trace(go.Scatter(x=d.index, y=d['bb_upper'], name='BB Upper', line=dict(color='rgba(55,71,79,0.6)', width=1)), row=1, col=1)
    fig.add_trace(go.Scatter(x=d.index, y=d['bb_lower'], name='BB Lower', line=dict(color='rgba(55,71,79,0.6)', width=1),
                             fill='tonexty', fillcolor='rgba(55,71,79,0.08)'), row=1, col=1)
    fig.add_trace(go.Scatter(x=d.index, y=d['bb_mid'], name='BB Mid', line=dict(color='rgba(55,71,79,0.4)', width=1, dash='dot')), row=1, col=1)
    # VWAP
    typical = (d['high'] + d['low'] + d['close']) / 3.0
    vwap = (typical * d['volume']).cumsum() / d['volume'].cumsum().replace(0, np.nan)
    fig.add_trace(go.Scatter(x=d.index, y=vwap, name='VWAP', line=dict(color='#8d6e63', width=2, dash='dot')), row=1, col=1)

    # Row 2: Volume
    fig.add_trace(go.Bar(x=d.index, y=d['volume'], marker_color=vol_colors, name='Volume', showlegend=False), row=2, col=1)

    # Row 3: RSI + MACD
    fig.add_trace(go.Scatter(x=d.index, y=d['rsi_14'], name='RSI 14', line=dict(color='#26c6da', width=1.5)), row=3, col=1)
    fig.add_hline(y=30, line=dict(color='rgba(239,83,80,0.5)', dash='dot'), row=3, col=1)
    fig.add_hline(y=70, line=dict(color='rgba(102,187,106,0.5)', dash='dot'), row=3, col=1)

    fig.add_trace(go.Bar(x=d.index, y=d['macd_hist'], name='MACD Hist', marker_color='rgba(96,125,139,0.6)'), row=3, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(x=d.index, y=d['macd'], name='MACD', line=dict(color='#607d8b', width=1.5)), row=3, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(x=d.index, y=d['macd_signal'], name='Signal', line=dict(color='#ff7043', width=1.5)), row=3, col=1, secondary_y=True)

    # Layout
    fig.update_layout(
        title=dict(
            text=f"SPY Intraday TA Dashboard — {date_str}",
            x=0.01, xanchor='left', y=0.98, yanchor='top',
            font=dict(size=18), pad=dict(b=10)
        ),
        template='plotly_white',
        legend=dict(
            orientation='h', x=0.01, xanchor='left', y=-0.18, yanchor='top',
            bgcolor='rgba(255,255,255,0.9)', bordercolor='rgba(0,0,0,0.12)', borderwidth=1,
            font=dict(size=11), itemwidth=40, groupclick='toggleitem'
        ),
        margin=dict(l=40, r=20, t=90, b=120),
        hovermode='x unified',
    )
    fig.update_xaxes(showgrid=True, gridcolor='rgba(200,200,200,0.2)', rangeslider_visible=False, type='date')
    fig.update_yaxes(showgrid=True, gridcolor='rgba(200,200,200,0.2)', title_text='Price', row=1, col=1)
    fig.update_yaxes(showgrid=True, gridcolor='rgba(200,200,200,0.2)', title_text='Volume', row=2, col=1)
    fig.update_yaxes(showgrid=True, gridcolor='rgba(200,200,200,0.2)', title_text='RSI', row=3, col=1)

    # RTH shading (09:30-16:00 ET)
    try:
        import pytz
        ny = pytz.timezone('America/New_York')
        start_rth = pd.Timestamp(date_str + ' 09:30', tz=ny).astimezone(pytz.UTC)
        end_rth   = pd.Timestamp(date_str + ' 16:00', tz=ny).astimezone(pytz.UTC)
        for r in (1, 2, 3):
            fig.add_vrect(x0=start_rth, x1=end_rth, fillcolor='rgba(33,150,243,0.08)', line_width=0, layer='below', row=r, col=1)
    except Exception:
        pass

    return fig

# Callbacks
def _render(_=None):
    with out:
        out.clear_output(wait=True)
        date_str = date_dd.value
        try:
            d = _load_day(date_str)
            if d is None or d.empty:
                status_lbl.value = f"No rows found for {date_str}. Check filenames/columns."
                print(status_lbl.value)
                return
            # Debug snapshot
            try:
                print(f"[debug] rows={len(d)} index_range=[{d.index.min()} .. {d.index.max()}] cols={list(d.columns)[:10]}")
            except Exception:
                pass
            fig = _build_fig(d, date_str)
            _last['fig'] = fig
            _last['date'] = date_str
            display(fig)
            status_lbl.value = f"Rendered {date_str} ({len(d):,} rows). Use the buttons to export."
        except Exception as e:
            status_lbl.value = f"Error rendering {date_str}: {e}"
            print(status_lbl.value)

def _export_html(_):
    if not _last['fig']:
        status_lbl.value = 'Render a chart first.'
        return
    os.makedirs('charts', exist_ok=True)
    path = os.path.join('charts', f"spy_intraday_{_last['date']}.html")
    pio.write_html(_last['fig'], file=path, auto_open=False, include_plotlyjs='cdn')
    status_lbl.value = f"Saved HTML: {path}"

def _export_png(_):
    if not _last['fig']:
        status_lbl.value = 'Render a chart first.'
        return
    os.makedirs('charts', exist_ok=True)
    path = os.path.join('charts', f"spy_intraday_{_last['date']}.png")
    try:
        _last['fig'].write_image(path, scale=2)
        status_lbl.value = f"Saved PNG: {path}"
    except Exception as e:
        status_lbl.value = f"PNG export failed. Ensure kaleido is installed. Error: {e}"

# Wire up widgets
date_dd.observe(lambda change: _render() if change['name'] == 'value' else None, names='value')
export_html_btn.on_click(_export_html)
export_png_btn.on_click(_export_png)

# UI
controls = widgets.HBox([date_dd, use_duckdb_chk, export_html_btn, export_png_btn])
display(widgets.VBox([controls, status_lbl, out]))

# Initial render
_render()

## Gold-Layer Data Visualization

This section loads and visualizes the gold-layer SPY data, which includes additional features like:
- Session indicators (RTH vs After Hours)
- 1-minute returns
- Standardized column names with `utc_timestamp`

The gold data should have enhanced analytics-ready features compared to the silver layer.

In [12]:
# Load and Inspect Gold-Layer SPY Data
import os
import pandas as pd

# Set up path to gold SPY data
gold_dir = os.path.join(
    os.path.dirname(os.path.dirname(os.path.abspath("../scripts/inspect_silver_spy.py"))),
    "data", "gold", "SPY"
)

print(f"Gold directory: {gold_dir}")

# Check if gold directory exists and list files
if os.path.exists(gold_dir):
    gold_files = sorted([f for f in os.listdir(gold_dir) if f.endswith(".parquet")])
    print(f"Found {len(gold_files)} gold parquet files")
    
    if gold_files:
        # Load a sample gold file
        sample_gold_file = os.path.join(gold_dir, gold_files[0])
        gold_df = pd.read_parquet(sample_gold_file)
        
        print(f"\nSample gold file: {sample_gold_file}")
        print(f"Shape: {gold_df.shape}")
        print(f"Columns: {gold_df.columns.tolist()}")
        
        # Display sample data
        display(gold_df.head())
        
        # Show info about the data
        display(gold_df.info())
        
        # Check unique session values
        if 'session' in gold_df.columns:
            print(f"\nSession values: {gold_df['session'].unique()}")
            print(f"Session counts:\n{gold_df['session'].value_counts()}")
        
        # Check return statistics
        if 'return_1min' in gold_df.columns:
            print(f"\nReturn statistics:")
            print(f"Mean: {gold_df['return_1min'].mean():.6f}")
            print(f"Std: {gold_df['return_1min'].std():.6f}")
            print(f"Min: {gold_df['return_1min'].min():.6f}")
            print(f"Max: {gold_df['return_1min'].max():.6f}")
    else:
        print("No gold parquet files found. You may need to run the silver-to-gold transformation script first.")
else:
    print("Gold directory does not exist. You may need to run the silver-to-gold transformation script first.")

Gold directory: /home/agoodman/workspace/antman_data/data/gold/SPY
Found 149 gold parquet files

Sample gold file: /home/agoodman/workspace/antman_data/data/gold/SPY/spy_1min_gold_2025-01-02.parquet
Shape: (391, 11)
Columns: ['utc_timestamp', 'et', 'session', 'open', 'high', 'low', 'close', 'volume', 'num_trades', 'vwap', 'return_1m']


,utc_timestamp,et,session,open,high,low,close,volume,num_trades,vwap,return_1m
0,2025-01-02 14:30:00+00:00,2025-01-02 09:30:00-05:00,RTH,589.39,589.45,588.4600,589.45,1300579,16472,588.918280,NaN
1,2025-01-02 14:31:00+00:00,2025-01-02 09:31:00-05:00,RTH,589.45,589.60,589.0400,589.57,175662,2160,589.334934,0.000204
2,2025-01-02 14:32:00+00:00,2025-01-02 09:32:00-05:00,RTH,589.49,589.65,589.0319,589.09,125032,1542,589.381349,-0.000814
3,2025-01-02 14:33:00+00:00,2025-01-02 09:33:00-05:00,RTH,589.09,589.18,587.9800,588.01,132101,2396,588.624495,-0.001833
4,2025-01-02 14:34:00+00:00,2025-01-02 09:34:00-05:00,RTH,587.99,588.52,587.8100,587.83,145261,1916,588.180358,-0.000306


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype                           
---  ------         --------------  -----                           
 0   utc_timestamp  391 non-null    datetime64[ns, UTC]             
 1   et             391 non-null    datetime64[ns, America/New_York]
 2   session        391 non-null    object                          
 3   open           391 non-null    float64                         
 4   high           391 non-null    float64                         
 5   low            391 non-null    float64                         
 6   close          391 non-null    float64                         
 7   volume         391 non-null    int64                           
 8   num_trades     391 non-null    int64                           
 9   vwap           391 non-null    float64                         
 10  return_1m      390 non-null    float64                        

None


Session values: ['RTH']
Session counts:
session
RTH    391
Name: count, dtype: int64


In [13]:
# Interactive Gold-Layer Dashboard + Technical Analysis (Mirrors Silver Chart)
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display

# Only proceed if we have gold data available
if 'gold_dir' in globals() and os.path.exists(gold_dir):
    gold_files = sorted([f for f in os.listdir(gold_dir) if f.endswith(".parquet")])
    
    if gold_files:
        # Collect available dates from gold files
        gold_available_dates = sorted({f.split('_')[-1].replace('.parquet', '') for f in gold_files})
        
        # Widgets for gold dashboard
        gold_date_dd = widgets.Dropdown(options=gold_available_dates, value=gold_available_dates[0], description='Gold Date:', layout=widgets.Layout(width='260px'))
        gold_export_html_btn = widgets.Button(description='Export HTML', tooltip='Save gold chart to charts/ as HTML', icon='download')
        gold_export_png_btn = widgets.Button(description='Export PNG', tooltip='Save gold chart to charts/ as PNG (requires kaleido)', icon='file')
        gold_status_lbl = widgets.Label('Select a date to render the gold dashboard...')
        gold_out = widgets.Output()
        
        _gold_last = {'fig': None, 'date': None}
        
        # Gold helpers
        def _gold_compute_indicators(d: pd.DataFrame) -> pd.DataFrame:
            d = d.copy()
            # EMAs
            for span in (9, 21, 50):
                d[f'ema_{span}'] = d['close'].ewm(span=span, adjust=False).mean()
            # Bollinger Bands (20, 2)
            mid = d['close'].rolling(20, min_periods=1).mean()
            std = d['close'].rolling(20, min_periods=1).std()
            d['bb_mid'] = mid
            d['bb_upper'] = mid + 2 * std
            d['bb_lower'] = mid - 2 * std
            # RSI(14) - Wilder
            delta = d['close'].diff()
            gain = np.where(delta > 0, delta, 0.0)
            loss = np.where(delta < 0, -delta, 0.0)
            roll_up = pd.Series(gain, index=d.index).ewm(alpha=1/14, adjust=False).mean()
            roll_down = pd.Series(loss, index=d.index).ewm(alpha=1/14, adjust=False).mean()
            rs = roll_up / (roll_down + 1e-9)
            d['rsi_14'] = 100 - (100 / (1 + rs))
            # MACD(12,26,9)
            ema12 = d['close'].ewm(span=12, adjust=False).mean()
            ema26 = d['close'].ewm(span=26, adjust=False).mean()
            d['macd'] = ema12 - ema26
            d['macd_signal'] = d['macd'].ewm(span=9, adjust=False).mean()
            d['macd_hist'] = d['macd'] - d['macd_signal']
            return d
        
        def _gold_load_day(date_str: str) -> pd.DataFrame:
            path = os.path.join(gold_dir, f"spy_1min_{date_str}.parquet")
            if not os.path.exists(path):
                raise FileNotFoundError(f"Missing gold parquet for {date_str}: {path}")
            d = pd.read_parquet(path)
            
            # Handle gold data structure (may have utc_timestamp column)
            if 'utc_timestamp' in d.columns:
                d = d.set_index('utc_timestamp')
            
            if not isinstance(d.index, pd.DatetimeIndex):
                d.index = pd.to_datetime(d.index, utc=True)
            elif d.index.tz is None:
                d.index = d.index.tz_localize('UTC')
            return d
        
        def _gold_build_fig(d: pd.DataFrame, date_str: str) -> go.Figure:
            d = _gold_compute_indicators(d)
            
            # Enhanced volume coloring by session if available
            if 'session' in d.columns:
                vol_colors = np.where(d['session'] == 'RTH', 'rgba(33,150,243,0.6)', 'rgba(255,152,0,0.6)')
            else:
                vol_colors = np.where(d['close'] >= d['open'], 'rgba(0,150,0,0.6)', 'rgba(200,0,0,0.6)')
            
            fig = make_subplots(
                rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.03,
                row_heights=[0.58, 0.18, 0.24],
                specs=[[{"secondary_y": False}],
                       [{"secondary_y": False}],
                       [{"secondary_y": True}]]
            )
            
            # Row 1: Price with overlays
            fig.add_trace(go.Candlestick(x=d.index, open=d['open'], high=d['high'], low=d['low'], close=d['close'],
                                         increasing_line_color='#26a69a', decreasing_line_color='#ef5350', name='OHLC',
                                         showlegend=False), row=1, col=1)
            for span, color in [(9, '#42a5f5'), (21, '#ab47bc'), (50, '#ffa726')]:
                fig.add_trace(go.Scatter(x=d.index, y=d[f'ema_{span}'], name=f'EMA {span}', line=dict(width=1.4, color=color)), row=1, col=1)
            # Bollinger
            fig.add_trace(go.Scatter(x=d.index, y=d['bb_upper'], name='BB Upper', line=dict(color='rgba(55,71,79,0.6)', width=1)), row=1, col=1)
            fig.add_trace(go.Scatter(x=d.index, y=d['bb_lower'], name='BB Lower', line=dict(color='rgba(55,71,79,0.6)', width=1),
                                     fill='tonexty', fillcolor='rgba(55,71,79,0.08)'), row=1, col=1)
            fig.add_trace(go.Scatter(x=d.index, y=d['bb_mid'], name='BB Mid', line=dict(color='rgba(55,71,79,0.4)', width=1, dash='dot')), row=1, col=1)
            # VWAP
            typical = (d['high'] + d['low'] + d['close']) / 3.0
            vwap = (typical * d['volume']).cumsum() / d['volume'].cumsum().replace(0, np.nan)
            fig.add_trace(go.Scatter(x=d.index, y=vwap, name='VWAP', line=dict(color='#8d6e63', width=2, dash='dot')), row=1, col=1)
            
            # Row 2: Volume with session highlighting
            fig.add_trace(go.Bar(x=d.index, y=d['volume'], marker_color=vol_colors, name='Volume', showlegend=False), row=2, col=1)
            
            # Row 3: RSI + MACD
            fig.add_trace(go.Scatter(x=d.index, y=d['rsi_14'], name='RSI 14', line=dict(color='#26c6da', width=1.5)), row=3, col=1)
            fig.add_hline(y=30, line=dict(color='rgba(239,83,80,0.5)', dash='dot'), row=3, col=1)
            fig.add_hline(y=70, line=dict(color='rgba(102,187,106,0.5)', dash='dot'), row=3, col=1)
            
            fig.add_trace(go.Bar(x=d.index, y=d['macd_hist'], name='MACD Hist', marker_color='rgba(96,125,139,0.6)'), row=3, col=1, secondary_y=True)
            fig.add_trace(go.Scatter(x=d.index, y=d['macd'], name='MACD', line=dict(color='#607d8b', width=1.5)), row=3, col=1, secondary_y=True)
            fig.add_trace(go.Scatter(x=d.index, y=d['macd_signal'], name='Signal', line=dict(color='#ff7043', width=1.5)), row=3, col=1, secondary_y=True)
            
            # Layout (matching silver chart style)
            fig.update_layout(
                title=dict(
                    text=f"SPY Gold-Layer TA Dashboard — {date_str}",
                    x=0.01, xanchor='left', y=0.98, yanchor='top',
                    font=dict(size=18), pad=dict(b=10)
                ),
                template='plotly_white',
                legend=dict(
                    orientation='h', x=0.01, xanchor='left', y=-0.18, yanchor='top',
                    bgcolor='rgba(255,255,255,0.9)', bordercolor='rgba(0,0,0,0.12)', borderwidth=1,
                    font=dict(size=11), itemwidth=40, groupclick='toggleitem'
                ),
                margin=dict(l=40, r=20, t=90, b=120),
                hovermode='x unified',
            )
            fig.update_xaxes(showgrid=True, gridcolor='rgba(200,200,200,0.2)', rangeslider_visible=False, type='date')
            fig.update_yaxes(showgrid=True, gridcolor='rgba(200,200,200,0.2)', title_text='Price', row=1, col=1)
            fig.update_yaxes(showgrid=True, gridcolor='rgba(200,200,200,0.2)', title_text='Volume', row=2, col=1)
            fig.update_yaxes(showgrid=True, gridcolor='rgba(200,200,200,0.2)', title_text='RSI', row=3, col=1)
            
            # RTH shading (09:30-16:00 ET)
            try:
                import pytz
                ny = pytz.timezone('America/New_York')
                start_rth = pd.Timestamp(date_str + ' 09:30', tz=ny).astimezone(pytz.UTC)
                end_rth   = pd.Timestamp(date_str + ' 16:00', tz=ny).astimezone(pytz.UTC)
                for r in (1, 2, 3):
                    fig.add_vrect(x0=start_rth, x1=end_rth, fillcolor='rgba(33,150,243,0.08)', line_width=0, layer='below', row=r, col=1)
            except Exception:
                pass
            
            return fig
        
        # Gold callbacks
        def _gold_render(_=None):
            with gold_out:
                gold_out.clear_output(wait=True)
                date_str = gold_date_dd.value
                try:
                    d = _gold_load_day(date_str)
                    fig = _gold_build_fig(d, date_str)
                    _gold_last['fig'] = fig
                    _gold_last['date'] = date_str
                    display(fig)
                    gold_status_lbl.value = f"Rendered gold data for {date_str}. Use the buttons to export."
                    
                    # Print gold-specific stats
                    if 'session' in d.columns:
                        session_stats = d['session'].value_counts()
                        print(f"Session breakdown: {dict(session_stats)}")
                    if 'return_1min' in d.columns:
                        returns = d['return_1min']
                        print(f"Return stats: Mean={returns.mean():.6f}, Std={returns.std():.6f}")
                        
                except Exception as e:
                    print(f"Error loading gold data for {date_str}: {e}")
                    gold_status_lbl.value = f"Error: {e}"
        
        def _gold_export_html(_):
            if not _gold_last['fig']:
                gold_status_lbl.value = 'Render a chart first.'
                return
            os.makedirs('charts', exist_ok=True)
            path = os.path.join('charts', f"spy_gold_intraday_{_gold_last['date']}.html")
            pio.write_html(_gold_last['fig'], file=path, auto_open=False, include_plotlyjs='cdn')
            gold_status_lbl.value = f"Saved HTML: {path}"
        
        def _gold_export_png(_):
            if not _gold_last['fig']:
                gold_status_lbl.value = 'Render a chart first.'
                return
            os.makedirs('charts', exist_ok=True)
            path = os.path.join('charts', f"spy_gold_intraday_{_gold_last['date']}.png")
            try:
                _gold_last['fig'].write_image(path, scale=2)
                gold_status_lbl.value = f"Saved PNG: {path}"
            except Exception as e:
                gold_status_lbl.value = f"PNG export failed. Ensure kaleido is installed. Error: {e}"
        
        # Wire up gold widgets
        gold_date_dd.observe(lambda change: _gold_render() if change['name'] == 'value' else None, names='value')
        gold_export_html_btn.on_click(_gold_export_html)
        gold_export_png_btn.on_click(_gold_export_png)
        
        # Gold UI
        gold_controls = widgets.HBox([gold_date_dd, gold_export_html_btn, gold_export_png_btn])
        display(widgets.VBox([gold_controls, gold_status_lbl, gold_out]))
        
        # Initial render
        _gold_render()
        
    else:
        print("No gold parquet files found. Run the silver-to-gold transformation script first.")
else:
    print("Gold directory not available. Please run the previous cell to set up gold_dir variable.")

## Before vs After: Loading and Summarizing Parquet (Pandas vs DuckDB)

This notebook originally used pandas to iterate over Parquet files for summaries and day loads. We’ve added DuckDB to push the heavy lifting down to a fast SQL engine over Parquet:

- Before (pandas):
  - Count rows: loop over files and `pd.read_parquet` each one, summing `len(df)`.
  - Load a day: read a specific file like `spy_1min_YYYY-MM-DD.parquet` with pandas.
- After (DuckDB):
  - Count rows: `SELECT COUNT(*) FROM read_parquet('.../*.parquet', union_by_name=true)` — no full DataFrame loads.
  - Load a day: filter by filename or by timestamp using a single SQL query and return a pandas DataFrame.

This change makes summary operations much faster and reduces memory usage. If pandas reads fail, ensure `pyarrow` is installed.

In [14]:
# Demo: Pandas vs DuckDB for SPY silver data
import os
import time
import pandas as pd
import duckdb

# Paths and file list assumed already defined earlier: silver_dir, files
if 'silver_dir' not in globals():
    raise RuntimeError("silver_dir not set; run earlier cells.")
if 'files' not in globals() or not files:
    files = sorted([f for f in os.listdir(silver_dir) if f.endswith('.parquet')])
    if not files:
        raise RuntimeError("No Parquet files found in silver_dir.")

some_date = files[0].split('_')[-1].replace('.parquet', '')  # pick first date from filenames
print(f"Using date for demo: {some_date}")

# 1) Count rows across all files
start = time.time()
rows_pandas = 0
for f in files:
    d = pd.read_parquet(os.path.join(silver_dir, f))
    rows_pandas += len(d)
print(f"Pandas count rows: {rows_pandas:,} rows in {time.time()-start:.2f}s")

start = time.time()
con = duckdb.connect()
(rows_duckdb,) = con.execute(
    "SELECT COUNT(*) FROM read_parquet(?, union_by_name=true)",
    [os.path.join(silver_dir, '*.parquet')]
).fetchone()
con.close()
print(f"DuckDB  count rows: {rows_duckdb:,} rows in {time.time()-start:.2f}s")

# 2) Load a single day
p_path = os.path.join(silver_dir, f"spy_1min_{some_date}.parquet")
start = time.time()
df_pandas = pd.read_parquet(p_path)
print(f"Pandas load day: {len(df_pandas):,} rows in {time.time()-start:.2f}s from {p_path}")

start = time.time()
con = duckdb.connect()
df_duck = con.execute(
    "SELECT * FROM read_parquet(?, union_by_name=true, filename=true) WHERE filename LIKE ?",
    [os.path.join(silver_dir, '*.parquet'), f"%{some_date}.parquet"]
).df()
con.close()
print(f"DuckDB  load day: {len(df_duck):,} rows in {time.time()-start:.2f}s (filtered by filename)")

Using date for demo: 2025-01-02
Pandas count rows: 58,094 rows in 0.91s
DuckDB  count rows: 58,094 rows in 0.04s
Pandas load day: 391 rows in 0.01s from /home/agoodman/workspace/antman_data/data/silver/SPY/spy_1min_2025-01-02.parquet
DuckDB  load day: 391 rows in 0.04s (filtered by filename)
Pandas count rows: 58,094 rows in 0.91s
DuckDB  count rows: 58,094 rows in 0.04s
Pandas load day: 391 rows in 0.01s from /home/agoodman/workspace/antman_data/data/silver/SPY/spy_1min_2025-01-02.parquet
DuckDB  load day: 391 rows in 0.04s (filtered by filename)
